In [1]:
from pathlib import Path
import json
from typing import List
from dataclasses import dataclass

# Tes dataclasses à adapter si besoin
@dataclass
class PointGPS:
    time: str
    lat: float
    lon: float
    alt: float
    dist: float
    hr: int
    speed: float
    cadence: int
    watts: int

@dataclass
class Lap:
    start_time: str
    total_time: float
    distance: float
    max_speed: float
    calories: int
    avg_hr: int
    max_hr: int
    intensity: str
    trigger: str
    points: List[PointGPS]

@dataclass
class Activite:
    sport: str
    id: str
    laps: List[Lap]

# Utilitaire pour désérialiser un fichier JSON en Activite
def load_activity(json_path: Path) -> Activite:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    laps = []
    for lap_data in data["laps"]:
        points = [PointGPS(**pt) for pt in lap_data["points"]]
        laps.append(Lap(**{**lap_data, "points": points}))

    return Activite(**{**data, "laps": laps})

# Charge toutes les activités
def load_all_activities(activities_dir: Path) -> List[Activite]:
    activities = []
    for json_file in activities_dir.glob("*.json"):
        if json_file.name == "index.json":
            continue
        activities.append(load_activity(json_file))
    return activities


In [4]:
all_activities = load_all_activities(Path("chemin/vers/activities_json"))

for act in all_activities:
    total_dist = sum(lap.distance for lap in act.laps) / 1000  # en km
    total_time = sum(lap.total_time for lap in act.laps) / 60  # en minutes
    print(f"{act.id} - {act.sport} : {total_dist:.2f} km en {total_time:.1f} min")


In [ ]:
#plot interactive route 

def plot_interactive_route(latitudes, longitudes):
    # Créer une carte centrée sur le premier point
    map_center = [latitudes[0], longitudes[0]]
    route_map = folium.Map(location=map_center, zoom_start=15)

    # Ajouter des points de passage
    points = list(zip(latitudes, longitudes))
    folium.PolyLine(points, color="blue", weight=5).add_to(route_map)

    # Ajouter des marqueurs pour le début et la fin du trajet
    folium.Marker(points[0], popup="Début", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(points[-1], popup="Fin", icon=folium.Icon(color="red")).add_to(route_map)

    # Sauvegarder la carte dans un fichier HTML
    route_map.save("route_map.html")
    print("Carte enregistrée : route_map.html")


In [ ]:
def plot_heatmap(all_coordinates):
    # Créer une carte centrée sur le premier point
    map_center = all_coordinates[0] if all_coordinates else [0, 0]
    heatmap = folium.Map(location=map_center, zoom_start=13)

    # Ajouter les données de la heatmap
    HeatMap(all_coordinates, radius=10, blur=15, max_zoom=1).add_to(heatmap)

    # Sauvegarder la carte
    heatmap.save("heatmap.html")
    print("Carte enregistrée : heatmap.html")
    return heatmap

# Créer une heatmap
plot_heatmap(all_coordinates)

In [ ]:
def stats_parcours(points):
    total_distance = sum(p.distance_from_prev for p in points)
    total_time = (points[-1].time - points[0].time).total_seconds()
    vitesse_moy = (total_distance / total_time * 3.6) if total_time > 0 else 0  # en km/h
    print(f"Distance totale : {total_distance:.1f} m")
    print(f"Durée totale    : {total_time/60:.1f} min")
    print(f"Vitesse moyenne : {vitesse_moy:.2f} km/h")

In [ ]:
def plot_vitesse_altitude(points):
    temps = [(p.time - points[0].time).total_seconds() / 60 for p in points]  # en minutes
    vitesses = [p.vitesse_kmh for p in points]
    altitudes = [p.elevation for p in points]

    fig, ax1 = plt.subplots(figsize=(10, 6))

    ax1.set_xlabel("Temps (minutes)")
    ax1.set_ylabel("Vitesse (km/h)", color="blue")
    ax1.plot(temps, vitesses, color="blue", label="Vitesse")
    ax1.tick_params(axis='y', labelcolor="blue")

    ax2 = ax1.twinx()
    ax2.set_ylabel("Altitude (m)", color="green")
    ax2.plot(temps, altitudes, color="green", linestyle="--", label="Altitude")
    ax2.tick_params(axis='y', labelcolor="green")

    plt.title("Évolution de la vitesse et de l'altitude")
    plt.grid(True)
    plt.show()

